In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectPercentile
from sklearn.svm import SVC
%matplotlib inline

import pingouin as pg
import researchpy   

df_train = pd.read_csv("Titanic_train.csv")
df_train['Survived_cate']=df_train['Survived']
#df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
print(df_train.info())

#取出資料後，把遺失值刪除
complete_data = df_train.dropna()
display(complete_data.head(5))

#排除 Ｎame 當特徵，先刪除每一個人都是獨特的資料，先檢視其他變數
complete_data = complete_data.drop(['Name', 'Ticket','PassengerId'], axis = 1)
display(complete_data.head(5))
print(complete_data.shape)


#Q1：目標變數為 Survived，以 Pclass, Age, SibSp, Parch, Fare,Sex, Embarked 為特徵，
#    試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。
num_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

cat_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'object':
        cat_features.append(feature)
print(f'{len(cat_features)} category Features : {cat_features}\n')

x=complete_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex', 'Embarked']]
y=complete_data['Survived']
display(x.head(10))
display(y.head(10))
#離散要轉換成數值，'Sex', 'Embarked'
sex_mapping = {
           'male': 1,
           'female': 0}
complete_data['Sex1'] = complete_data['Sex'].map(sex_mapping)
embarked_mapping = {
           'C': 0,
           'Q': 1,
           'S': 2}
complete_data['Embarked1'] = complete_data['Embarked'].map(embarked_mapping)
display(complete_data.head(10))
#執行 包裝法
x = complete_data[['Sex1','Embarked1','Pclass', 'Age', 'SibSp','Parch','Fare']]
display(x)
y = complete_data['Survived_cate']
estimator = SVC(kernel="linear")
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(x, y)
## True= selected feature
print(selector.support_)
#Selected (i.e., estimated best) features are assigned rank 1.
ranking=selector.ranking_
print(ranking)
rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived_cate
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,0
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S,1
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,1


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived_cate
1,1,1,female,38.0,1,0,71.2833,C85,C,1
3,1,1,female,35.0,1,0,53.1000,C123,S,1
6,0,1,male,54.0,0,0,51.8625,E46,S,0
10,1,3,female,4.0,1,1,16.7000,G6,S,1
11,1,1,female,58.0,0,0,26.5500,C103,S,1


(183, 10)
7 Numeric Features : ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived_cate']

3 category Features : ['Sex', 'Cabin', 'Embarked']



,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,female,C
3,1,35.0,1,0,53.1000,female,S
6,1,54.0,0,0,51.8625,male,S
10,3,4.0,1,1,16.7000,female,S
11,1,58.0,0,0,26.5500,female,S
21,2,34.0,0,0,13.0000,male,S
23,1,28.0,0,0,35.5000,male,S
27,1,19.0,3,2,263.0000,male,S
52,1,49.0,1,0,76.7292,female,C
54,1,65.0,0,1,61.9792,male,C


1     1
3     1
6     0
10    1
11    1
21    1
23    1
27    0
52    1
54    0
Name: Survived, dtype: int64

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived_cate,Sex1,Embarked1
1,1,1,female,38.0,1,0,71.2833,C85,C,1,0,0
3,1,1,female,35.0,1,0,53.1000,C123,S,1,0,2
6,0,1,male,54.0,0,0,51.8625,E46,S,0,1,2
10,1,3,female,4.0,1,1,16.7000,G6,S,1,0,2
11,1,1,female,58.0,0,0,26.5500,C103,S,1,0,2
21,1,2,male,34.0,0,0,13.0000,D56,S,1,1,2
23,1,1,male,28.0,0,0,35.5000,A6,S,1,1,2
27,0,1,male,19.0,3,2,263.0000,C23 C25 C27,S,0,1,2
52,1,1,female,49.0,1,0,76.7292,D33,C,1,0,0
54,0,1,male,65.0,0,1,61.9792,B30,C,0,1,0


,Sex1,Embarked1,Pclass,Age,SibSp,Parch,Fare
1,0,0,1,38.0,1,0,71.2833
3,0,2,1,35.0,1,0,53.1000
6,1,2,1,54.0,0,0,51.8625
10,0,2,3,4.0,1,1,16.7000
11,0,2,1,58.0,0,0,26.5500
...,...,...,...,...,...,...,...
871,0,2,1,47.0,1,1,52.5542
872,1,2,1,33.0,0,0,5.0000
879,0,0,1,56.0,0,1,83.1583
887,0,2,1,19.0,0,0,30.0000


[ True False False False False False False]
[1 4 3 5 2 6 7]
['Sex1']
